In [2]:
!pip install pyrankvote

  Using cached https://files.pythonhosted.org/packages/a4/f5/834b6c81681bf0037092cf37bb43adfa5e966d98a498fb0b3309c1f0e13f/pyrankvote-2.0.1-py3-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [78]:
import pyrankvote
import ast
import pandas as pd
from pyrankvote import Candidate, Ballot


def rename_index_col_to_ballot_id(df):
    df.rename(columns={'Unnamed: 0': 'ballot_id'}, inplace=True)
    return df

def get_cand_list(df):
    list_of_unique_vals_in_dataframe = df.iloc[:, 1].value_counts().index
    cand_list = list(filter(lambda x: x != '0', list_of_unique_vals_in_dataframe))
    return cand_list

def initialize_cand_objs(cand_list):
    if isinstance(cand_list, str):
        return [Candidate(c) for c in cand_list.split(', ')]
    else:
        return [Candidate(c) for c in cand_list]

# def initialize_cand_objs_in_df_cols(df):
#     cands_df = df.iloc[:, 1:]  # getting all columns with candidate names in them
#     cands_df = cands_df.applymap(lambda x: create_cand(x))  # making them into pyrankvote candidate objects
#     cands_df['ballot_id'] = df.iloc[:, 0]  # stitching df back with ballot_ids from old dataframe
#     return cands_df

def get_cands_into_single_cell(df):
    df['candidate_list'] = df.iloc[:, 1:].agg(", ".join, axis=1)
    # cands_to_put_in_single_cell = df.iloc[:, 1:].columns.to_list()
    # df['candidate_list'] = str(cands_to_put_in_single_cell)
    # df['candidate_list'] = df['candidate_list'].apply(lambda x: ast.literal_eval(x))
    return df

def initialize_ballot_objs(df):
    ballot_objects = []
    for index, value in enumerate(df['candidate_list']):
        ballot = Ballot(ranked_candidates=value)
        ballot_objects.append(ballot)
    return ballot_objects

def run_election(list_of_cand_objs, election_df):
    return pyrankvote.instant_runoff_voting(list_of_cand_objs, election_df['ballots'], pick_random_if_blank=True)

def rm_invalid_rows(df):
    return df[df['candidate_list'] != '0']

def run_all_steps(df):
    #df = pd.read_csv('/Users/shvarugh/Documents/myGit/ranked-choice-voting/master_elections.csv')
#     df = rename_index_col_to_ballot_id(df)
    df = df.dropna(axis=1, how='all')
    df = df.drop(['ballot_id','filename'], axis=1)
    print(df.shape)
    df = get_cands_into_single_cell(df)

    df['candidate_list'] = df['candidate_list'].apply(lambda x: x.replace('0, ', '').replace(', 0', ''))
    df = rm_invalid_rows(df)
    df['candidate_list'] = df['candidate_list'].apply(lambda x: initialize_cand_objs(x))

    ballots = initialize_ballot_objs(df)
    df['ballots'] = ballots

    cand_list = get_cand_list(df)
    cand_list = initialize_cand_objs(cand_list)
    print("cand_list: ",cand_list)
    print(df.head())
    election = run_election(cand_list, df)
    print("get_winners",election.get_winners())
    print("rounds",type(election.rounds))
#     open file and write results. read winners votes from file and write to df
    print("register_round_results",election.register_round_results(election.rounds[-1]))

    print("election:", election)


    print('hi')







In [57]:
df = pd.read_csv('/Users/shvarugh/Documents/myGit/ranked-choice-voting/master_elections.csv')
df = rename_index_col_to_ballot_id(df)

/Users/shvarugh/.pyenv/versions/jupyter3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [58]:
pd.set_option('max_colwidth', 8000)
pd.options.display.max_columns = None
pd.options.display.float_format = '{:.4f}'.format
pd.options.display.max_rows = 10000

In [59]:
df[df['filename']=="election_05-04-2020_08-40-23_5cands_16noise.csv"]

,ballot_id,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,candidate_6,filename
0,0,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
1,1,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
2,2,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
3,3,candidate_1,candidate_2,candidate_3,0,0,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
4,4,candidate_1,candidate_2,candidate_3,candidate_4,candidate_5,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
...,...,...,...,...,...,...,...,...
41358,41358,candidate_5,candidate_4,candidate_3,candidate_2,candidate_1,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
41359,41359,candidate_5,candidate_4,candidate_3,candidate_2,candidate_1,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
41360,41360,candidate_5,candidate_4,candidate_3,candidate_2,candidate_1,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv
41361,41361,candidate_5,candidate_4,candidate_3,candidate_2,candidate_1,NaN,election_05-04-2020_08-40-23_5cands_16noise.csv


In [60]:
election_ids= df['filename'].unique()

In [79]:
for election_num, election in enumerate(election_ids):
    #print(election_num, election)
    if election_num == 0:
        df_election = df.loc[df['filename']==election]
        print(df_election.shape)
        run_all_steps(df_election)

(41363, 8)
(41363, 5)
cand_list:  [<Candidate('candidate_1')>, <Candidate('candidate_5')>, <Candidate('candidate_2')>, <Candidate('candidate_3')>, <Candidate('candidate_4')>]
   candidate_1  candidate_2  candidate_3  candidate_4  candidate_5  \
0  candidate_1  candidate_2  candidate_3  candidate_4  candidate_5   
1  candidate_1  candidate_2  candidate_3  candidate_4  candidate_5   
2  candidate_1  candidate_2  candidate_3  candidate_4  candidate_5   
3  candidate_1  candidate_2  candidate_3            0            0   
4  candidate_1  candidate_2  candidate_3  candidate_4  candidate_5   

                                         candidate_list  \
0  [candidate_2, candidate_3, candidate_4, candidate_5]   
1  [candidate_2, candidate_3, candidate_4, candidate_5]   
2  [candidate_2, candidate_3, candidate_4, candidate_5]   
3                            [candidate_2, candidate_3]   
4  [candidate_2, candidate_3, candidate_4, candidate_5]   

                                                 

In [3]:
main()

DuplicateCandidatesError: 

In [4]:
df = pd.read_csv('./data/election_05-02-2020_08-17-43_3cands_11noise.csv')
df = rename_index_col_to_ballot_id(df)
df = get_cands_into_single_cell(df)
df['candidate_list'] = df['candidate_list'].apply(lambda x: x.replace('0, ','').replace(', 0',''))

df.head()

,ballot_id,candidate_1,candidate_2,candidate_3,candidate_list
0,0,candidate_1,candidate_2,candidate_3,"candidate_1, candidate_2, candidate_3"
1,1,0,0,candidate_3,candidate_3
2,2,candidate_1,candidate_2,candidate_3,"candidate_1, candidate_2, candidate_3"
3,3,candidate_1,candidate_2,candidate_3,"candidate_1, candidate_2, candidate_3"
4,4,candidate_1,0,candidate_3,"candidate_1, candidate_3"


In [6]:
df['candidate_list'] = df['candidate_list'].apply(lambda x: initialize_cand_objs(x))
df.head()

,ballot_id,candidate_1,candidate_2,candidate_3,candidate_list
0,0,candidate_1,candidate_2,candidate_3,"[candidate_1, candidate_2, candidate_3]"
1,1,0,0,candidate_3,[candidate_3]
2,2,candidate_1,candidate_2,candidate_3,"[candidate_1, candidate_2, candidate_3]"
3,3,candidate_1,candidate_2,candidate_3,"[candidate_1, candidate_2, candidate_3]"
4,4,candidate_1,0,candidate_3,"[candidate_1, candidate_3]"


In [7]:
ballots = initialize_ballot_objs(df)
df['ballots'] = ballots

In [ ]:
df.head()

In [ ]:
cand_list = get_cand_list(df)
df.head()

In [ ]:
cand_list

In [ ]:
cand_list = initialize_cand_objs(cand_list)
cand_list

In [ ]:
pd.set_option('display.max_rows', 500)


In [ ]:
pd.set_option('display.max_rows', len(df))

In [ ]:
df

In [ ]:
import itertools
set(itertools.chain.from_iterable(df['candidate_list']))

In [ ]:
pyrankvote.instant_runoff_voting(cand_list, df['ballots'])

In [ ]:
election = run_election(cand_list, df)
print(election)


#print('hi')






# if __name__ == "__main__":
#     main()